In [0]:
!nvidia-smi

In [0]:
iteration = 300
batchsize = 16
snapshot_interval = 20
display_interval = 20
update_interval = 20
phase = 0
'''
0- : all decoder training.
1- : trans encoding.(hold)
'''

In [3]:
mount = '/content/gdrive'

from google.colab import drive
drive.mount(mount)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy
import math
import glob
from PIL import Image, ImageOps, ImageChops, ImageFilter
import matplotlib.pyplot as plt
import cupy

import chainer
from chainer import training, backend, Variable
from chainer.training import extensions
import chainer.functions as F
import chainer.links as L
import chainer.backends.cuda

from IPython.display import clear_output

In [0]:
def gaussian(size):
    return F.gaussian(cupy.zeros(size, dtype=cupy.float32),
                     cupy.ones(size, dtype=cupy.float32))

In [0]:
def make_optimizer_Adam(model, alpha=1e-4, beta1=0.9, clip=True):
    optimizer = chainer.optimizers.Adam(alpha=alpha, beta1=beta1)
    optimizer.setup(model)
    if clip:
        optimizer.add_hook(chainer.optimizer_hooks.GradientClipping(1.))
    return optimizer

In [0]:
def out_generated_image(gen, c_iter, v_iter):
    @chainer.training.make_extension()
    def make_image(trainer):
        clear_output()
        xp = gen.xp
        c_base = xp.asarray(c_iter.next())
        z_c = Variable(c_base[:,1])/255. *2. -1.
        if phase < 7:
            v_base = c_base[:,0]
        else:
            v_base = xp.asarray(v_iter.next())[:,0]
        z_v = Variable(v_base)/255. *2. -1.
        
        with chainer.using_config('train', False):
            x, _ = gen(z_c, z_v)
        x = F.transpose(F.reshape(x, (-1, 3, 128, 128)), (0, 2, 3, 1))
        x = chainer.backends.cuda.to_cpu(x.array)
        
        plt.figure(figsize=(16, 12))
        
        
        for i, img in enumerate(xp.asnumpy(c_base[:,1])):
            plt.subplot(3,4,i+1)
            plt.axis('off')
            plt.imshow(Image.fromarray(numpy.uint8(img).transpose((1, 2, 0))))
            
        for i, img in enumerate(x):
            plt.subplot(3,4,i+5)
            plt.axis('off')
            plt.imshow(Image.fromarray(numpy.uint8((img+1.)/2. *255.)))
            
        for i, img in enumerate(xp.asnumpy(v_base)):
            plt.subplot(3,4,i+9)
            plt.axis('off')
            plt.imshow(Image.fromarray(numpy.uint8(img).transpose((1, 2, 0))))
        
        plt.show()
    return make_image

In [0]:
class DiscriminatorGen(chainer.Chain):

    def __init__(self, wscale=1.):
        super(DiscriminatorGen, self).__init__()
        
        with self.init_scope():
            w = chainer.initializers.HeNormal(wscale)
            
            self.c0_0 = L.Convolution2D(3, 32, 3, 1, 1, initialW=w)
            self.bn0_0 = L.BatchNormalization(32)
            self.c0_1 = L.Convolution2D(None, 32, 3, 1, 1, initialW=w)
            self.bn0_1 = L.BatchNormalization(32)
            
            self.c1_0 = L.Convolution2D(None, 64, 1, 1, 0, initialW=w)
            self.bn1_0 = L.BatchNormalization(64)
            self.c1_1 = L.Convolution2D(None, 64, 1, 1, 0, initialW=w)
            self.bn1_1 = L.BatchNormalization(64)
            self.c1_2 = L.Convolution2D(None, 64, 1, 1, 0, initialW=w)
            self.bn1_2 = L.BatchNormalization(64)
            self.c1_3 = L.Convolution2D(None, 64, 3, 1, 1, initialW=w)
            self.bn1_3 = L.BatchNormalization(64)
            self.c1_4 = L.Convolution2D(None, 64, 3, 1, 1, initialW=w)
            self.bn1_4 = L.BatchNormalization(64)
            
            self.c2_0 = L.Convolution2D(None, 128, 3, 1, 1, initialW=w)
            self.bn2_0 = L.BatchNormalization(128)
            self.c2_1 = L.Convolution2D(None, 128, 3, 1, 1, initialW=w)
            self.bn2_1 = L.BatchNormalization(128)
            
            self.c3_0 = L.Convolution2D(None, 256, 3, 1, 1, initialW=w)
            self.bn3_0 = L.BatchNormalization(256)
            self.c3_1 = L.Convolution2D(None, 256, 3, 1, 1, initialW=w)
            self.bn3_1 = L.BatchNormalization(256)
            
            self.c4_0 = L.Convolution2D(None, 512, 3, 1, 1, initialW=w)
            self.bn4_0 = L.BatchNormalization(512)
            self.c4_1 = L.Convolution2D(None, 512, 3, 1, 1, initialW=w)
            self.bn4_1= L.BatchNormalization(512)
            
            self.c5_0 = L.Convolution2D(None, 512, 3, 1, 1, initialW=w)
            self.bn5_0 = L.BatchNormalization(512)
            self.c5_1 = L.Convolution2D(None, 512, 3, 1, 1, initialW=w)
            self.bn5_1 = L.BatchNormalization(512)
            
            self.l_m = L.Linear(None, 1024, initialW=w)
            self.t = L.Linear(None, 64*16, initialW=None)
            
            self.l_o = L.Linear(None, 1, initialW=w)

    def minibatch_discrimination(self, m):
        batchsize = m.shape[0]
        m = F.expand_dims(m, 3)
        m_T = F.transpose(m, (3, 1, 2, 0))
        m, m_T = F.broadcast(m, m_T)
        norm = F.sum(F.absolute_error(m, m_T), axis=2)
        eraser = F.broadcast_to(cupy.eye(batchsize, dtype=cupy.float32).reshape((batchsize, 1, batchsize)), norm.shape)
        c_b = F.exp(-(norm + 1e6 * eraser))
        o_b = F.sum(c_b, axis=2)
        return o_b
        
    def gaussian(self, x):
        return F.gaussian(x, cupy.ones(x.shape, dtype=cupy.float32)*2e-2)
    
    def self_attention(self, x, f, g, h, gamma=0.01):
        f_shape = f.shape
        g_shape = g.shape
        h_shape = h.shape
        
        attention_map = f*F.transpose(g, (0, 1, 3, 2))
        feature_map = h*attention_map
        return F.add(x, feature_map*gamma)
    
    def gap(self, x):
        return F.average_pooling_2d(x, x.shape[3])
    
    def forward(self, z):
        batchsize = z.shape[0]
        x = F.leaky_relu(self.bn0_0(self.c0_0(self.gaussian(z))))
        _x = F.leaky_relu(self.bn0_1(self.c0_1(self.gaussian(x))))
        x = F.concat((x, _x))
        # 128
        x = F.max_pooling_2d(x, 2)
        f = F.tanh(self.bn1_0(self.c1_0(x)))
        g = F.tanh(self.bn1_1(self.c1_1(x)))
        h = F.tanh(self.bn1_2(self.c1_2(x)))
        x = F.leaky_relu(self.self_attention(x, f, g, h))
        x = F.leaky_relu(self.bn1_3(self.c1_3(self.gaussian(x))))
        _x = F.leaky_relu(self.bn1_4(self.c1_4(self.gaussian(x))))
        x = F.concat((x, _x))
        # 64
        x = F.max_pooling_2d(x, 2)
        x = F.leaky_relu(self.bn2_0(self.c2_0(self.gaussian(x))))
        _x = F.leaky_relu(self.bn2_1(self.c2_1(self.gaussian(x))))
        x = F.concat((x, _x))
        # 32
        x = F.max_pooling_2d(x, 2)
        x = F.leaky_relu(self.bn3_0(self.c3_0(self.gaussian(x))))
        _x = F.leaky_relu(self.bn3_1(self.c3_1(self.gaussian(x))))
        x = F.concat((x, _x))
        # 16
        x = F.max_pooling_2d(x, 2)
        x = F.leaky_relu(self.bn4_0(self.c4_0(self.gaussian(x))))
        _x = F.leaky_relu(self.bn4_1(self.c4_1(self.gaussian(x))))
        x = F.concat((x, _x))
        # 8
        x = F.max_pooling_2d(x, 2)
        x = F.leaky_relu(self.bn5_0(self.c5_0(self.gaussian(x))))
        _x = F.leaky_relu(self.bn5_1(self.c5_1(self.gaussian(x))))
        x = F.concat((x, _x))
        x = self.gap(x)
        # 4
        
        h_m = F.leaky_relu(self.l_m(x))
        m = F.reshape(self.t(h_m), (batchsize, 64, 16))
        o_b = self.minibatch_discrimination(m)
        
        x = F.concat((h_m, o_b), axis=1)
        
        return self.l_o(x), h_m

In [0]:
class DiscriminatorEn(chainer.Chain):

    def __init__(self, wscale=1.):
        super(DiscriminatorEn, self).__init__()
        
        with self.init_scope():
            w = chainer.initializers.HeNormal(wscale)
            
            self.l0 = L.Linear(None, 1024, initialW=w)
            self.l1 = L.Linear(None, 1024, initialW=w)
            self.t = L.Linear(None, 64*16, initialW=None)
            
            self.l_o = L.Linear(None, 1, initialW=w)

    def minibatch_discrimination(self, m):
        batchsize = m.shape[0]
        m = F.expand_dims(m, 3)
        m_T = F.transpose(m, (3, 1, 2, 0))
        m, m_T = F.broadcast(m, m_T)
        norm = F.sum(F.absolute_error(m, m_T), axis=2)
        eraser = F.broadcast_to(cupy.eye(batchsize, dtype=cupy.float32).reshape((batchsize, 1, batchsize)), norm.shape)
        c_b = F.exp(-(norm + 1e6 * eraser))
        o_b = F.sum(c_b, axis=2)
        return o_b
        
    def forward(self, z):
        batchsize = z.shape[0]
        h = F.relu(self.l0(z))
        h = F.relu(self.l1(h))
        m = F.reshape(self.t(h), (batchsize, 64, 16))
        o_b = self.minibatch_discrimination(m)
        
        out = F.concat((h, o_b), axis=1)
        
        return self.l_o(out)

In [0]:
class Generator(chainer.Chain):

    def __init__(self, wscale=1.):
        super(Generator, self).__init__()

        with self.init_scope():
            w = chainer.initializers.HeNormal(wscale)
            
            self.stc0_0 = L.Convolution2D(3, 32, 3, 1, 1, initialW=w)
            self.stbn0_0 = L.BatchNormalization(32)
            self.stc0_1 = L.Convolution2D(None, 32, 3, 1, 1, initialW=w)
            self.stbn0_1 = L.BatchNormalization(32)
            
            self.stc1_0 = L.Convolution2D(None, 64, 1, 1, 0, initialW=w)
            self.stbn1_0 = L.BatchNormalization(64)
            self.stc1_1 = L.Convolution2D(None, 64, 1, 1, 0, initialW=w)
            self.stbn1_1 = L.BatchNormalization(64)
            self.stc1_2 = L.Convolution2D(None, 64, 1, 1, 0, initialW=w)
            self.stbn1_2 = L.BatchNormalization(64)
            self.stc1_3 = L.Convolution2D(None, 64, 3, 1, 1, initialW=w)
            self.stbn1_3 = L.BatchNormalization(64)
            self.stc1_4 = L.Convolution2D(None, 64, 3, 1, 1, initialW=w)
            self.stbn1_4 = L.BatchNormalization(64)
            
            self.stc2_0 = L.Convolution2D(None, 128, 3, 1, 1, initialW=w)
            self.stbn2_0 = L.BatchNormalization(128)
            self.stc2_1 = L.Convolution2D(None, 128, 3, 1, 1, initialW=w)
            self.stbn2_1 = L.BatchNormalization(128)
            
            self.stc3_0 = L.Convolution2D(None, 256, 3, 1, 1, initialW=w)
            self.stbn3_0 = L.BatchNormalization(256)
            self.stc3_1 = L.Convolution2D(None, 256, 3, 1, 1, initialW=w)
            self.stbn3_1 = L.BatchNormalization(256)
            
            self.stc4_0 = L.Convolution2D(None, 512, 3, 1, 1, initialW=w)
            self.stbn4_0 = L.BatchNormalization(512)
            self.stc4_1 = L.Convolution2D(None, 512, 3, 1, 1, initialW=w)
            self.stbn4_1 = L.BatchNormalization(512)
            
            self.stc5_0 = L.Convolution2D(None, 512, 3, 1, 1, initialW=w)
            self.stbn5_0 = L.BatchNormalization(512)
            self.stc5_1 = L.Convolution2D(None, 512, 3, 1, 1, initialW=w)
            self.stbn5_1 = L.BatchNormalization(512)
            
            self.stl1 = L.Linear(None, 1024, initialW=w)
            self.stl2 = L.Linear(None, 1024, initialW=w)
            self.stl3 = L.Linear(None, 1024, initialW=w)
            
            self.stbtm = L.Linear(None, 512*8*8, initialW=w)
            
            self.enc0_0 = L.Convolution2D(None, 32, 1, 1, 0, initialW=w)
            self.enbnc0_0 = L.BatchNormalization(32)
            self.enc0_1 = L.Convolution2D(None, 32, 3, 1, 1, initialW=w)
            self.enbnc0_1 = L.BatchNormalization(32)
            
            self.enc1_0 = L.Convolution2D(None, 128, 1, 1, 0, initialW=w)
            self.enbnc1_0 = L.BatchNormalization(128)
            self.enc1_1 = L.Convolution2D(None, 128, 3, 1, 1, initialW=w)
            self.enbnc1_1 = L.BatchNormalization(128)
            
            self.enc2_0 = L.Convolution2D(None, 128, 1, 1, 0, initialW=w)
            self.enbnc2_0 = L.BatchNormalization(128)
            self.enc2_1 = L.Convolution2D(None, 128, 3, 1, 1, initialW=w)
            self.enbnc2_1 = L.BatchNormalization(128)
            
            self.enc3_0 = L.Convolution2D(None, 128, 1, 1, 0, initialW=w)
            self.enbnc3_0 = L.BatchNormalization(128)
            self.enc3_1 = L.Convolution2D(None, 128, 3, 1, 1, initialW=w)
            self.enbnc3_1 = L.BatchNormalization(128)

            self.enc_gamma = L.Convolution2D(None, 128, 1, 1, 0, initialW=w)
            self.enbnc_gamma = L.BatchNormalization(128)
            self.enc_beta = L.Convolution2D(None, 128, 1, 1, 0, initialW=w)
            self.enbnc_beta = L.BatchNormalization(128)
            
            self.de5_0 = L.Convolution2D(None, 512, 3, 1, 1, initialW=w)
            self.debn5_0 = L.BatchNormalization(512)
            self.de5_1 = L.Convolution2D(None, 512, 3, 1, 1, initialW=w)
            self.debn5_1 = L.BatchNormalization(512)
            
            self.de4_0 = L.Convolution2D(None, 256, 3, 1, 1, initialW=w)
            self.debn4_0 = L.BatchNormalization(256)
            self.de4_1 = L.Convolution2D(None, 256, 3, 1, 1, initialW=w)
            self.debn4_1 = L.BatchNormalization(256)
            
            self.de3_0 = L.Convolution2D(None, 128, 3, 1, 1, initialW=w)
            self.debn3_0 = L.BatchNormalization(128)
            self.de3_1 = L.Convolution2D(None, 128, 3, 1, 1, initialW=w)
            self.debn3_1 = L.BatchNormalization(128)
            
            self.de2_0 = L.Convolution2D(None, 64, 3, 1, 1, initialW=w)
            self.debn2_0 = L.BatchNormalization(64)
            self.de2_1 = L.Convolution2D(None, 64, 3, 1, 1, initialW=w)
            self.debn2_1 = L.BatchNormalization(64)
            self.de2_2 = L.Convolution2D(None, 128, 1, 1, 0, initialW=w)
            self.debn2_2 = L.BatchNormalization(128)
            self.de2_3 = L.Convolution2D(None, 128, 1, 1, 0, initialW=w)
            self.debn2_3 = L.BatchNormalization(128)
            
            self.de1_0 = L.Convolution2D(None, 32, 3, 1, 1, initialW=w)
            self.debn1_0 = L.BatchNormalization(32)
            self.de1_1 = L.Convolution2D(None, 32, 3, 1, 1, initialW=w)
            self.debn1_1 = L.BatchNormalization(32)
            
            self.c_out = L.Convolution2D(None, 3, 1, 1, 0, initialW=w)
            
    def self_attention(self, x, f, g, h, gamma=0.01):
        f_shape = f.shape
        g_shape = g.shape
        h_shape = h.shape
        
        attention_map = f*F.transpose(g, (0, 1, 3, 2))
        feature_map = h*attention_map
        return F.add(x, feature_map*gamma)
    
    def gap(self, x):
        return F.average_pooling_2d(x, x.shape[3])
    
    def spade(self, feature, gamma, beta):
        x = feature*gamma
        x = F.add(x, beta)
        return x
    
    def gaussian(self, batch, ch, w, h):
        size = batch*ch*w*h
        noise = F.gaussian(cupy.zeros(size, dtype=cupy.float32),
                           cupy.ones(size, dtype=cupy.float32))
        return F.reshape(noise, (batch, ch, w, h))
    
    def forward(self, x, style):        
        batch = x.shape[0]
        # style
        z = F.relu(self.stbn0_0(self.stc0_0(style)))
        _z = F.relu(self.stbn0_1(self.stc0_1(z)))
        z = F.concat((z, _z))
        # 128
        z = F.max_pooling_2d(z, 2)
        f = F.tanh(self.stbn1_0(self.stc1_0(z)))
        g = F.tanh(self.stbn1_1(self.stc1_1(z)))
        h = F.tanh(self.stbn1_2(self.stc1_2(z)))
        z = F.relu(self.self_attention(z, f, g, h))
        z = F.relu(self.stbn1_3(self.stc1_3(z)))
        _z = F.relu(self.stbn1_4(self.stc1_4(z)))
        z = F.concat((z,_z))
        # 64
        z = F.max_pooling_2d(z, 2)
        z = F.relu(self.stbn2_0(self.stc2_0(z)))
        _z = F.relu(self.stbn2_1(self.stc2_1(z)))
        z = F.concat((z, _z))
        # 32
        z = F.max_pooling_2d(z, 2)
        z = F.relu(self.stbn3_0(self.stc3_0(z)))
        _z = F.relu(self.stbn3_1(self.stc3_1(z)))
        z = F.concat((z, _z))
        # 16
        z = F.max_pooling_2d(z, 2)
        z = F.relu(self.stbn4_0(self.stc4_0(z)))
        _z = F.relu(self.stbn4_1(self.stc4_1(z)))
        z = F.concat((z, _z))
        # 8
        z = F.max_pooling_2d(z, 2)
        z = F.relu(self.stbn5_0(self.stc5_0(z)))
        _z = F.relu(self.stbn5_1(self.stc5_1(z)))
        z = F.concat((z, _z))
        # 4
        z = self.gap(z)
        
        z = F.relu(self.stl1(z))
        z = F.relu(self.stl2(z))
        z = F.relu(self.stl3(z))
        
        # Encode
        s = F.relu(self.enbnc0_0(self.enc0_0(x)))
        f = F.relu(self.enbnc0_1(self.enc0_1(x)))
        # 128
        s_64 = F.space2depth(s, 2)
        f = F.space2depth(f, 2)
        s_64 = F.relu(self.enbnc1_0(self.enc1_0(s_64)))
        f = F.relu(self.enbnc1_1(self.enc1_1(f)))
        # 64
        s_32 = F.average_pooling_2d(s_64, 2)
        f = F.max_pooling_2d(f, 2)
        s_32 = F.relu(self.enbnc2_0(self.enc2_0(s_32)))
        f = F.relu(self.enbnc2_1(self.enc2_1(f)))
        # 32
        s = F.average_pooling_2d(s_32, 2)
        f = F.max_pooling_2d(f, 2)
        s = F.relu(self.enbnc3_0(self.enc3_0(s)))
        f = F.relu(self.enbnc3_1(self.enc3_1(f)))
        f = F.relu(f*s)
        # 16
        f = F.unpooling_2d(f, 2, cover_all=False)
        f = F.relu(f*s_32)
        # 32
        f = F.unpooling_2d(f, 2, cover_all=False)
        f = F.relu(f*s_64)
        g = F.relu(self.enbnc_gamma(self.enc_gamma(f)))
        b = F.relu(self.enbnc_beta(self.enc_beta(f)))
        # 64
        
        # Decode
        h = F.relu(F.reshape(self.stbtm(z), (-1, 512, 8, 8)))
        n = self.gaussian(batch, 1, 8, 8)
        h = F.concat((h, n))
        h = F.relu(self.debn5_0(self.de5_0(h)))
        n = self.gaussian(batch, 1, 8, 8)
        _h = F.concat((h, n))
        _h = F.relu(self.debn5_1(self.de5_1(_h)))
        h = F.concat((h, _h))
        # 8
        h = F.depth2space(h, 2)
        n = self.gaussian(batch, 1, 16, 16)
        h = F.concat((h, n))
        h = F.relu(self.debn4_0(self.de4_0(h)))
        n = self.gaussian(batch, 1, 16, 16)
        _h = F.concat((h, n))
        _h = F.relu(self.debn4_1(self.de4_1(_h)))
        h = F.concat((h, _h))
        # 16
        h = F.depth2space(h, 2)
        n = self.gaussian(batch, 1, 32, 32)
        h = F.concat((h, n))
        h = F.relu(self.debn3_0(self.de3_0(h)))
        n = self.gaussian(batch, 1, 32, 32)
        _h = F.concat((h, n))
        _h = F.relu(self.debn3_1(self.de3_1(_h)))
        h = F.concat((h, _h))
        # 32
        h = F.depth2space(h, 2)
        n = self.gaussian(batch, 1, 64, 64)
        h = F.concat((h, n))
        h = F.relu(self.debn2_0(self.de2_0(h)))
        n = self.gaussian(batch, 1, 64, 64)
        _h = F.concat((h, n))
        _h = F.relu(self.debn2_1(self.de2_1(_h)))
        h = F.concat((h, _h))
        h = F.relu(self.debn2_2(self.de2_2(h)))
        h = F.relu(self.spade(h, g, b))
        h = F.relu(self.debn2_3(self.de2_3(h)))
        # 64
        h = F.depth2space(h, 2)
        n = self.gaussian(batch, 1, 128, 128)
        h = F.concat((h, n))
        h = F.relu(self.debn1_0(self.de1_0(h)))
        n = self.gaussian(batch, 1, 128, 128)
        _h = F.concat((h, n))
        _h = F.relu(self.debn1_1(self.de1_1(_h)))
        h = F.concat((h, _h))
        
        h = F.tanh(self.c_out(h))
        # 128
        
        return h, z

In [0]:
class DCGANUpdater(chainer.training.updaters.StandardUpdater):

    def __init__(self, *args, **kwargs):
        self.gen, self.dis_gen, self.dis_en = kwargs.pop('models')
        self.cache_fake = None
        self.cache_dis_gen = None
        self.cache_dis_en = None
        self.k = 5
        super(DCGANUpdater, self).__init__(*args, **kwargs)

    def loss_dis(self, dis, ll, opt):
        loss = sum(ll)
        chainer.report({opt: loss}, dis)
        return loss
    
    def loss_en(self, en, ll):
        loss = sum(ll) 
        chainer.report({'loss': loss, 'style_rec': ll[-1]}, en)
        return loss
    
    def loss_gen(self, gen, ll):
        loss = sum(ll) 
        chainer.report({'loss': loss, 'fake': ll[0], 'rec': ll[-1]}, gen)
        return loss

    def update_core(self):
        gen_optimizer = self.get_optimizer('gen')
        dis_gen_optimizer = self.get_optimizer('dis_gen')
        dis_en_optimizer = self.get_optimizer('dis_en')
        
        gen, dis_gen, dis_en = self.gen, self.dis_gen, self.dis_en
        main_iter = self.get_iterator('main')
        style_iter = self.get_iterator('style')
        
        for _ in range(self.k):
            batch_c = main_iter.next()
            batchsize = len(batch_c)
            x_c = Variable(self.converter(batch_c, device=self.device)) /255. *2. -1.
            x_c_real = x_c[:,0]
            x_c_base = x_c[:,1]

            batch_v = style_iter.next()
            x_v_real = Variable(self.converter(batch_v, device=self.device))[:,0] /255. *2. -1.
            
            if phase >= 1:
                x_fake, h_v_real = gen(x_c_base, x_v_real)
            else:
                x_fake, h_v_real = gen(x_c_base, x_c_real)
            
            y_c_real, fm_c_real = dis_gen(x_c_real)
            dis_loss_real = F.sum((y_c_real-1)**2) / batchsize
            dis_gen_optimizer.update(self.loss_dis, dis_gen, [dis_loss_real], 'c_real')

            y_v_real, fm_v_real = dis_gen(x_v_real)
            dis_loss_real = F.sum((y_v_real-1)**2) / batchsize
            dis_gen_optimizer.update(self.loss_dis, dis_gen, [dis_loss_real], 'v_real')
            
            y_fake, fm_fake = dis_gen(x_fake)
            dis_loss_fake = F.sum(y_fake**2) / batchsize
            dis_gen_optimizer.update(self.loss_dis, dis_gen, [dis_loss_fake], 'fake')
        
            if self.cache_fake is not None:
                remind, _ = dis_gen(self.cache_fake)
                dis_loss_remind = F.sum(remind**2) / batchsize
                dis_gen_optimizer.update(self.loss_dis, dis_gen, [dis_loss_remind], 'remind')

            self.cache_fake = x_fake
                
            if self.cache_dis_gen is None:
                self.cache_dis_gen = dis_gen.copy('copy')
                
        dis_gen.copyparams(self.cache_dis_gen)
        self.cache_dis_gen = None
        
        fake_loss = F.sum((y_fake-1)**2) / batchsize
            
        if phase < 1:
            fm_loss = F.mean_squared_error(fm_c_real, fm_fake)
            reconst_loss = F.mean_squared_error(x_c_real, x_fake)
        else:
            fm_loss = F.mean_squared_error(fm_v_real, fm_fake)
            reconst_loss = F.mean_squared_error(x_c_base, x_fake)

        gen_optimizer.update(self.loss_gen, gen, [fake_loss, fm_loss, reconst_loss])

In [0]:
GPU = 0
OUT = mount+'/My Drive/result/'
IMG_SHAPE = (128, 128)

In [0]:
dataset = mount+'/My Drive/picture/train_pic/**/*.jpg'

In [0]:
gen = Generator()
dis_gen = DiscriminatorGen()

In [15]:
chainer.backends.cuda.get_device_from_id(GPU).use()
gen.to_gpu()
dis_gen.to_gpu()

In [0]:
opt_gen = make_optimizer_Adam(gen, clip=False)
opt_dis_gen = make_optimizer_Adam(dis_gen, clip=False)
dis_gen.t.disable_update()

In [0]:
def crop_sq(img):
    img_width, img_height = img.size
    sub = img_width - img_height
    if sub == 0:
        return img
    crop_size = min(img.size)
    crop_position = numpy.random.randint(0, abs(sub))
    if sub > 0:
        return img.crop((crop_position, 0, crop_size+crop_position, crop_size))
    else:
        return img.crop((0, crop_position, crop_size, crop_size+crop_position))

In [0]:
def img_convert(img_array):
    img = Image.fromarray(numpy.uint8(img_array.transpose(1, 2, 0))).convert('RGB')
    width, height = img.size
    image_trans_trigger = numpy.random.randn(2)
    
    img = crop_sq(img)
        
    if image_trans_trigger[0] > 0:
        img = ImageOps.mirror(img)
        
    if image_trans_trigger[1] > 0:
        img = img.resize(IMG_SHAPE)
    else:
        img = img.resize((IMG_SHAPE[0]+16, IMG_SHAPE[1]+16))
        crop_position = numpy.random.randint(0, 16, 2)
        img = img.crop((crop_position[0],
                       crop_position[1],
                       crop_position[0]+IMG_SHAPE[0],
                       crop_position[1]+IMG_SHAPE[1]))

    real_array = numpy.asarray(img, dtype=numpy.float32).transpose(2, 0, 1)
    
    img = img.filter(ImageFilter.GaussianBlur(1.0))
    img = img.filter(ImageFilter.SMOOTH_MORE)
    img = ImageOps.equalize(img)
    img = img.quantize(8, kmeans=True).convert('RGB')
    
    base_array = numpy.asarray(img, dtype=numpy.float32).transpose(2, 0, 1)
    
    out_array = numpy.concatenate((real_array[None, :, :, :],
                                   base_array[None, :, :, :]), axis=0)
    return out_array

In [0]:
def img_resize(img_array):
    img = Image.fromarray(numpy.uint8(img_array.transpose(1, 2, 0))).convert('RGB')
    img = crop_sq(img)
      
    img = img.resize(IMG_SHAPE)
    
    real_array = numpy.asarray(img, dtype=numpy.float32).transpose(2, 0, 1)
    
    img = img.filter(ImageFilter.GaussianBlur(1.0))
    img = img.filter(ImageFilter.SMOOTH_MORE)
    img = ImageOps.equalize(img)
    img = img.quantize(8, kmeans=True).convert('RGB')
    
    base_array = numpy.asarray(img, dtype=numpy.float32).transpose(2, 0, 1)
    
    out_array = numpy.concatenate((real_array[None, :, :, :],
                                   base_array[None, :, :, :]), axis=0)
    return out_array

In [20]:
image_files = glob.glob(dataset, recursive=True)
print('{} contains {} image files'
      .format(dataset, len(image_files)))
img_dataset = chainer.datasets.ImageDataset(paths=image_files)

train = chainer.datasets.TransformDataset(img_dataset, img_convert)
sample = chainer.datasets.TransformDataset(img_dataset, img_resize)

/content/gdrive/My Drive/picture/train_pic/**/*.jpg contains 1062026 image files


In [0]:
train_c_iter = chainer.iterators.SerialIterator(train, batchsize, shuffle=True)
train_v_iter = chainer.iterators.SerialIterator(train, batchsize, shuffle=True)

In [0]:
updater = DCGANUpdater(
    models=(gen, dis_gen),
    iterator={'main': train_v_iter, 'style': train_c_iter},
    optimizer={'gen': opt_gen, 'dis_gen': opt_dis_gen},
    device=GPU)

In [0]:
trainer = training.Trainer(updater, (iteration, 'iteration'), out=OUT)

In [0]:
snapshot_interval = (snapshot_interval, 'iteration')
display_interval = (display_interval, 'iteration')

In [0]:
trainer.extend(extensions.LogReport(trigger=display_interval))
trainer.extend(out_generated_image(gen,
                                   chainer.iterators.SerialIterator(sample, 4, shuffle=True),
                                   chainer.iterators.SerialIterator(sample, 4, shuffle=True)),
               trigger=snapshot_interval)
trainer.extend(extensions.PrintReport([
    'epoch', 'iteration', 'dis_gen/c_real', 'dis_gen/fake', 'gen/loss'
    ]), trigger=display_interval)
trainer.extend(extensions.ProgressBar(update_interval=update_interval))

In [0]:
if phase > 0:
    chainer.serializers.load_npz(OUT+'korosu_gen.npz', gen, strict=False)
    chainer.serializers.load_npz(OUT+'korosu_dis_gen.npz', dis_gen, strict=False)
    chainer.serializers.load_npz(OUT+'korosu_opt_gen.npz', opt_gen, strict=False)
    chainer.serializers.load_npz(OUT+'korosu_opt_dis_gen.npz', opt_dis_gen, strict=False)

In [0]:
trainer.run()

In [0]:
chainer.serializers.save_npz(OUT+'korosu_gen.npz', gen)
chainer.serializers.save_npz(OUT+'korosu_dis_gen.npz', dis_gen)
chainer.serializers.save_npz(OUT+'korosu_opt_gen.npz', opt_gen)
chainer.serializers.save_npz(OUT+'korosu_opt_dis_gen.npz', opt_dis_gen)